In [1]:
import numpy as np
import matplotlib.pyplot as plt
import csv

Extrayendo columnas de los archivos de Yahoo Finance:

In [4]:
archivosYahoo = ['csvs/AVH.csv']#, 'csvs/CIB.csv', 'csvs/EC.csv']

In [7]:
for archivo in archivosYahoo:
    lector_csv = csv.reader(open(archivo))
    for renglon in lector_csv:
        print(renglon)

['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
['2017-10-01', '7.800000', '7.940000', '7.070000', '7.070000', '6.440903', '1648700']
['2017-11-01', '7.050000', '7.890000', '6.730000', '7.700000', '7.114349', '2116500']
['2017-12-01', '7.700000', '8.290000', '7.560000', '8.030000', '7.419249', '1484200']
['2018-01-01', '7.960000', '8.470000', '7.740000', '8.180000', '7.557841', '1311500']
['2018-02-01', '8.100000', '9.760000', '7.790000', '9.350000', '8.638853', '5483000']
['2018-03-01', '9.380000', '9.740000', '8.450000', '8.470000', '7.825784', '3098600']
['2018-04-01', '8.430000', '9.300000', '8.030000', '8.170000', '7.548601', '2136400']
['2018-05-01', '8.170000', '8.250000', '7.420000', '7.540000', '6.966519', '2196800']
['2018-06-01', '7.550000', '7.760000', '6.340000', '6.460000', '5.968662', '3514400']
['2018-07-01', '6.410000', '6.840000', '6.180000', '6.520000', '6.024098', '1597000']
['2018-08-01', '6.570000', '6.640000', '5.270000', '5.460000', '5.152653', '

Extrayendo columnas de los archivos de Investing.com

In [6]:
 archivosInv = ['csvs/ARG-2.csv', 'csvs/CFV-2.csv', 'csvs/CRT-2.csv', 'csvs/IMI-2.csv']